# Cumulative Prospect Theory

Part of this is reproduction from: https://towardsdatascience.com/an-introduction-to-bayesian-inference-in-pystan-c27078e58d53

Related documentation:
* https://mc-stan.org/docs/2_23/stan-users-guide/example-decision-analysis.html
* https://mc-stan.org/docs/2_25/stan-users-guide/examples.html

In [1]:
import pystan
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import multiprocessing
from scipy.special import expit

In [2]:
multiprocessing.set_start_method("fork")
sns.set()  # Nice plot aesthetic
np.random.seed(101)

## Generate data

In [3]:
N = 100
p = np.random.random(size=N)
x0 = np.random.random(size=N)
x1 = np.random.random(size=N)

risk = 0.3
inv_tmp = 0.1

v0 = p * x0**(1-risk)
v1 = p * x1**(1-risk)
delta = v1 - v0
p_choice1 = expit(delta)

y = np.zeros(N, dtype=int)
y[:] = p_choice1 > np.random.random(size=N)


data = {'N': N, 'x0': x1, 'x1': x1, 'p':p, 'y': list(y)}

In [4]:
model = """
functions {
    real eu(real x0, real x1, real p, int y, real alpha) {
    vector[2] v;
    vector[2] p_choice;
    int c;
    v[1] = p * pow(x0, 1-alpha);
    v[2] = (1- p) * pow(x1, 1-alpha);
    p_choice = softmax(v);
    c = y+1;
    return p_choice[c];
  }
}
data {
    int<lower=0> N;
    vector[N] x0;
    vector[N] x1;
    vector[N] p;
    int<lower=0,upper=1> y[N];
}
parameters {
    real<lower=-1, upper=1>alpha;
}
model {
    for (i in 1:N)
        target += log(eu(x0[i], x1[i], p[i], y[i], alpha));
}
"""

In [ ]:
# Compile the model
sm = pystan.StanModel(model_code=model)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f4328c65a1528dd4038c1e21f566b0c3 NOW.


In [ ]:
# Train the model and generate samples
fit = sm.sampling(data=data, iter=1000, chains=4, warmup=500, thin=1, seed=101)
print(fit)